In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet, Lasso
from statsmodels.tsa.holtwinters import ExponentialSmoothing, Holt, SimpleExpSmoothing
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, GRU, LSTM, Bidirectional
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

# Load the dataset
data = pd.read_csv('CSCO.csv')

# Preprocess the data (Assuming 'Date' and 'Target' columns are present)
data['Date'] = pd.to_datetime(data['Date'])
data = data.set_index('Date')
# If your data is not already sorted by date, you should sort it here
data = data.sort_index()

# Feature and target separation
# Assuming the target variable is the last column
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

# Data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling features (needed for certain ML models like KNN, SVM, MLP)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Dictionary to hold models and their names
models = {
    'AdaBoost': AdaBoostRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'Random Forest': RandomForestRegressor(),
    'Extra Trees': ExtraTreesRegressor(),
    'Bagging': BaggingRegressor(),
    'KNN (k=10)': KNeighborsRegressor(n_neighbors=10),
    'MLP': MLPRegressor(max_iter=1000),
    'KNN (k=5)': KNeighborsRegressor(n_neighbors=5),
    'KNN (k=3)': KNeighborsRegressor(n_neighbors=3),
    'Decision Tree': DecisionTreeRegressor(),
    'SVR (RBF)': SVR(kernel='rbf'),
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Elastic Net': ElasticNet(),
    'Lasso Regression': Lasso(),
    'SVR (Poly)': SVR(kernel='poly'),
    'SVR (Linear)': SVR(kernel='linear')
}

# Time series models like ETS require univariate data
# For simplicity, let's assume we're dealing with univariate time series forecasting
# This assumes that 'y' is a univariate time series

# Fit and predict with each model
results = {}
for name, model in models.items():
    if 'KNN' in name or 'SVR' in name or 'MLP' in name:
        model.fit(X_train_scaled, y_train)
        predictions = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    results[name] = mse
    print(f'{name} MSE: {mse}')

# ETS models
ets_models = {
    'Simple ETS': SimpleExpSmoothing,
    'Double ETS (Holt)': Holt,
    'Triple ETS (Holt-Winters)': ExponentialSmoothing
}

# Fit and predict with ETS models
for name, ets_model in ets_models.items():
    if name == 'Triple ETS (Holt-Winters)':
        model = ets_model(y_train, seasonal='add', seasonal_periods=12).fit()
    else:
        model = ets_model(y_train).fit()
    predictions = model.forecast(len(y_test))
    mse = mean_squared_error(y_test, predictions)
    results[name] = mse
    print(f'{name} MSE: {mse}')

# Deep Learning models require reshaping the data for time series forecasting
# This is a complex process and would require further details about the dataset
# Here is an example of how to define and train a simple LSTM model

def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        LSTM(50),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Reshaping the data for LSTM (this is a simplified example and may not work directly with your dataset)
# Assuming 'X_train_scaled' and 'X_test_scaled' are already scaled and are in 2D shape
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Build and train the LSTM model
lstm_model = build_lstm_model((X_train_reshaped.shape[1], 1))
lstm_model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32)

# Predict and evaluate LSTM model
lstm_predictions = lstm_model.predict(X_test_reshaped)
lstm_mse = mean_squared_error(y_test, lstm_predictions)
results['LSTM'] = lstm_mse
print(f'LSTM MSE: {lstm_mse}')

# You would need to follow a similar process to create and train CNN, GRU, and Bidirectional LSTM models

# Print all results
for model_name, mse in results.items():
    print(f'{model_name} - MSE: {mse}')
